In [1]:
import numpy as np

import pymc as pm
from aesara.graph import FunctionGraph, optimize_graph
from aesara.tensor.basic_opt import ShapeFeature, topo_constant_folding

import aesara
import aesara.tensor as at

from aeppl import factorized_joint_logprob

In [2]:
with pm.Model() as model:
    grw = pm.GaussianRandomWalk(
        name="grw", 
        mu=0, 
        sigma=1, 
        init_dist=pm.StudentT.dist(5), 
        steps=10, 
        size=None,
    )
    
    trace = pm.sample(chains=1)

ValueError: Random variables detected in the logp graph: [normal_rv{0, (0, 0), floatX, False}.out].
This can happen when DensityDist logp or Interval transform functions reference nonlocal variables.

In [3]:
aesara.dprint(grw)

CumOp{-1, add} [id A] 'grw'
 |Join [id B]
   |TensorConstant{0} [id C]
   |InplaceDimShuffle{x} [id D]
   | |studentt_rv{0, (0, 0, 0), floatX, False}.1 [id E]
   |   |RandomGeneratorSharedVariable(<Generator(PCG64) at 0x16B689120>) [id F]
   |   |TensorConstant{[]} [id G]
   |   |TensorConstant{11} [id H]
   |   |TensorConstant{5.0} [id I]
   |   |TensorConstant{0} [id J]
   |   |TensorConstant{1.0} [id K]
   |normal_rv{0, (0, 0), floatX, False}.1 [id L]
     |RandomGeneratorSharedVariable(<Generator(PCG64) at 0x16B689900>) [id M]
     |SpecifyShape [id N]
     | |Elemwise{Cast{int64}} [id O]
     | | |TensorConstant{(1,) of 10} [id P]
     | |TensorConstant{1} [id Q]
     |TensorConstant{11} [id R]
     |InplaceDimShuffle{x} [id S]
     | |TensorConstant{0.0} [id T]
     |InplaceDimShuffle{x} [id U]
       |TensorConstant{1.0} [id V]


In [4]:
aesara.dprint(pm.logp(grw, at.ones(11)))

Join [id A]
 |TensorConstant{0} [id B]
 |Check{lam > 0, nu > 0} [id C]
 | |Elemwise{sub,no_inplace} [id D]
 | | |Elemwise{sub,no_inplace} [id E]
 | | | |Elemwise{add,no_inplace} [id F]
 | | | | |Elemwise{gammaln,no_inplace} [id G]
 | | | | | |Elemwise{true_div,no_inplace} [id H]
 | | | | |   |Elemwise{add,no_inplace} [id I]
 | | | | |   | |TensorConstant{(1,) of 5.0} [id J]
 | | | | |   | |InplaceDimShuffle{x} [id K]
 | | | | |   |   |TensorConstant{1.0} [id L]
 | | | | |   |InplaceDimShuffle{x} [id M]
 | | | | |     |TensorConstant{2.0} [id N]
 | | | | |Elemwise{mul,no_inplace} [id O]
 | | | |   |InplaceDimShuffle{x} [id P]
 | | | |   | |TensorConstant{0.5} [id Q]
 | | | |   |Elemwise{log,no_inplace} [id R]
 | | | |     |Elemwise{true_div,no_inplace} [id S]
 | | | |       |Elemwise{pow,no_inplace} [id T]
 | | | |       | |Check{sigma > 0} [id U]
 | | | |       | | |TensorConstant{(1,) of 1.0} [id V]
 | | | |       | | |All [id W]
 | | | |       | |   |MakeVector{dtype='bool'} [id X]
 